## (1) Importing Libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
import numpy as np
import pandas as pd

In [3]:
import time
from multiprocessing import Pool

In [4]:
import sqlite3

## (2) Webscrapping

### (2.1) Scrapping only one page containing 20 books

In [5]:
# (i) Identify the website, open and extract the info (soup) within it: 
URL = "https://books.toscrape.com/catalogue/page-"
req = requests.get(URL+str(1)+'.html')
soup = BeautifulSoup(req.content, "html.parser")

In [6]:
# (ii) Extract the needed info nested in the li section with an specific class:
info_li = soup.find_all('li', attrs={'class': 'col-xs-6 col-sm-4 col-md-3 col-lg-3'})

In [7]:
# (iii) Validating the 20 books info was extracted:
len(info_li)

20

In [8]:
# (iv) Extract the section where the website and the titles are:
# Option A:
titles_ini = []
for i in range(0,len(info_li)):
    title = soup.find_all('li', class_= 'col-xs-6 col-sm-4 col-md-3 col-lg-3')[i].find("article", class_="product_pod").find("h3").find("a")
    titles_ini.append(title)
# Option B: 
#titles_ini = []
#for i in range(0,len(info_li)):
#    title = soup.find_all('li', class_= 'col-xs-6 col-sm-4 col-md-3 col-lg-3')[i].find("article", class_="product_pod").find("h3").find("a", title=True)
#    titles_ini.append(title)  

# Validating:
# titles_ini
# A list with the website and the full name of the book.

In [9]:
# (iv) Extracting the book full name:
title_final = []
for item in titles_ini:
    #print( item['title'])
    title_final.append(item['title'])
# title_final    

In [10]:
# (v) Extracting the book reference (website):
title_ref = []
for item in titles_ini:
    #print( item['href'])
    title_ref.append(item['href'])
# title_ref[0]  

In [11]:
# (vi) Extracting the category:
URL2 = "https://books.toscrape.com/catalogue/"
req2 = requests.get(URL2+title_ref[0])
soup2 = BeautifulSoup(req2.text, "html.parser")
category = soup2.find('ul', class_ = 'breadcrumb')#.findAll('li')#.find("a", href = True)
text = list(category.descendants)
text[16]

'Poetry'

In [12]:
# (vii) Another way to do Name Scraping:
Name = soup2.find('div', class_ = 'col-sm-6 product_main')
text = list(Name.descendants)
Name_final = (text[2])

In [13]:
# (viii) Extracting UPC:
UPC_Avai_Review = soup2.find('table', class_ = 'table table-striped')
text = list(UPC_Avai_Review.descendants)
print(text[6], text[47], text[56], text[22])

a897fe39b1053632 In stock (22 available) 0 Â£51.77


In [14]:
# (ix) Extracting the Availability:
Avai_num = int(re.findall(r'\d+', text[47])[0])
# Avai_num

In [15]:
# (x) text line finder: 
count = 0
for i in range(0,len(text)):
    count = 1 + count
    if text[i] == 'Â£51.77':
       print(count-1)
       break    

22


In [16]:
# (xi) Rating Scraping:
rating = soup2.find('div', class_= 'col-sm-6 product_main').find("p").find_next("p").find_next("p", class_= True)
# text = list(rating.descendants)
temp = rating['class']
rating_final = temp[1]
rating_final

'Three'

### (2.2) Scraping all 50 webpages

In [17]:
URL = "https://books.toscrape.com/catalogue/page-"
URL2 = "https://books.toscrape.com/catalogue/"

# (i) main variables
title_final = []
title_ref = []
rating_final = []
#instock_final = []
price_final = []
price_currency_final = []

# (ii) temporal variables
temp = []

# (iii) Webscraping algorithm:
a = 51
for page in range (1,a):
    req = requests.get(URL+str(page)+'.html')
    soup = BeautifulSoup(req.text, "html.parser")
    info_li = soup.find_all('li', attrs={'class': 'col-xs-6 col-sm-4 col-md-3 col-lg-3'})
    len(info_li)
    if page < a:
       titles_ini = []
       rating_ini = []
              
       for i in range(0,len(info_li)):
           title = soup.find_all('li', class_= 'col-xs-6 col-sm-4 col-md-3 col-lg-3')[i].find("article", class_="product_pod").find("h3").find("a")
           titles_ini.append(title)
           
           rating = soup.find_all('li', class_= 'col-xs-6 col-sm-4 col-md-3 col-lg-3')[i].find("article", class_="product_pod").find("p")
           rating_ini.append(rating)
           
           #instock = soup.find_all('li', class_= 'col-xs-6 col-sm-4 col-md-3 col-lg-3')[i].find("article", class_="product_pod").find('div', class_="product_price").find("p", class_="instock availability").get_text()
           #instock_final.append(re.findall(("In stock"), instock)[0])
                      
           price = soup.find_all('li', class_= 'col-xs-6 col-sm-4 col-md-3 col-lg-3')[i].find("article", class_="product_pod").find('div', class_="product_price").find("p", class_="price_color").get_text()
           price_currency = re.findall(("\D"), price)
           price_currency_final.append(price_currency[1])
           price = re.findall(("([0-9]+[,.]+[0-9]+)"), price)
           price_final.append(float(price[0]))
                  
       for item in titles_ini:
            title_final.append(item['title']) 
       for item in titles_ini:
            title_ref.append(item['href'])
       for item in rating_ini:
            temp = item['class']
            rating_final.append([temp[1]][0])
            #rating_final.append([temp[index] for index in [1]][0])

In [18]:
category_final = []
UPC_final = []
Avai_final = []
Review_final =[]

for i in range(0, len(title_ref)):      
    req2 = requests.get(URL2+title_ref[i])
    soup2 = BeautifulSoup(req2.text, "html.parser")
    
    # Category Scraping:
    category = soup2.find('ul', class_ = 'breadcrumb')#.findAll('li')#.find("a", href = True)
    text = list(category.descendants)
    category_final.append(text[16])
    
    # UPC, Availability and Review Scraping:
    UPC_Avai_Review = soup2.find('table', class_ = 'table table-striped')
    text = list(UPC_Avai_Review.descendants)
    UPC_final.append(text[6]) 
    Avai_final.append(int((re.findall(r'\d+', text[47]))[0]))
    Review_final.append(int(re.findall(r'\d+',text[56])[0]))

In [19]:
for i in range(0,len(rating_final)):
    if rating_final[i] == 'One':
        rating_final[i] = rating_final[i].replace('One', '1')
    elif rating_final[i] == 'Two':
        rating_final[i] = rating_final[i].replace('Two', '2')
    elif rating_final[i] ==  'Three':
        rating_final[i] = rating_final[i].replace('Three', '3')   
    elif rating_final[i] ==  'Four':
        rating_final[i] = rating_final[i].replace('Four', '4') 
    else:
        rating_final[i] = rating_final[i].replace('Five', '5')  
    rating_final[i] = int(rating_final[i])   

# Validating:    
# print(rating_final)    

In [20]:
# Validating:
# UPC_final
# category_final
# Avai_final
# Review_final

In [21]:
print(len(title_final), len(title_ref),
      len(rating_final),
      len(price_final),
      len(price_currency_final),
      len(category_final), len(UPC_final), len(Avai_final), len(Review_final))

1000 1000 1000 1000 1000 1000 1000 1000 1000


## (3) Data Frame

In [50]:
# data ={'Name': title_final, 'Rating': rating_final, 'Price Currency': price_currency_final,'Price': price_final,
#        'UPC': UPC_final, '# Available': Avai_final, '# of Review': Review_final, 'Category':category_final}
data ={'Name': title_final, 'Rating': rating_final, 'Price Currency': price_currency_final,'Price': price_final,
       'UPC': UPC_final, 'Available': Avai_final, 'Review': Review_final, 'Category':category_final}
DF =  pd.DataFrame(data)

In [51]:
#DF = DF.style.set_properties(**{'text-align': 'left'})
#display(DF)
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Name            1000 non-null   object 
 1   Rating          1000 non-null   int64  
 2   Price Currency  1000 non-null   object 
 3   Price           1000 non-null   float64
 4   UPC             1000 non-null   object 
 5   Available       1000 non-null   int64  
 6   Review          1000 non-null   int64  
 7   Category        1000 non-null   object 
dtypes: float64(1), int64(3), object(4)
memory usage: 62.6+ KB


In [52]:
DF.describe()

,Rating,Price,Available,Review
count,1000.000000,1000.00000,1000.000000,1000.0
mean,2.923000,35.07035,8.585000,0.0
std,1.434967,14.44669,5.654622,0.0
min,1.000000,10.00000,1.000000,0.0
25%,2.000000,22.10750,3.000000,0.0
50%,3.000000,35.98000,7.000000,0.0
75%,4.000000,47.45750,14.000000,0.0
max,5.000000,59.99000,22.000000,0.0


In [25]:
# DF.to_csv(r'C:/Users/luisc/Downloads/2022_ARCH_Partners/DF.csv', index=False, header=True)

## (4) Multiprocessing

In [26]:
# Version 1:
# Function parse:
# def parse(URL):
            
#     # Request and parsing the URL:      
#     req2 = requests.get(URL)
#     soup2 = BeautifulSoup(req2.text, "html.parser")
    
#     # Name Scraping:
#     Name = soup2.find('div', class_ = 'col-sm-6 product_main')
#     text = list(Name.descendants)
#     Name_final = (text[2])[0]
    
#     # Rating Scraping:
#     rating = soup2.find('div', class_= 'col-sm-6 product_main').find("p").find_next("p").find_next("p", class_= True)
#     temp = rating['class']
#     Rating_final = temp[1]
#     if Rating_final == 'One':
#         Rating_final = Rating_final.replace('One', '1')
#     elif Rating_final == 'Two':
#         Rating_final = Rating_final.replace('Two', '2')
#     elif Rating_final ==  'Three':
#         Rating_final = Rating_final.replace('Three', '3')   
#     elif Rating_final ==  'Four':
#         Rating_final = Rating_final.replace('Four', '4') 
#     else:
#         Rating_final = Rating_final.replace('Five', '5')  
    
#     Rating_final = int(Rating_final)  
    
    
#     # UPC, Availability and Review Scraping:
#     UPC_Avai_Review_Price = soup2.find('table', class_ = 'table table-striped')
#     text = list(UPC_Avai_Review_Price.descendants)
#     UPC_final = text[6] 
#     Avai_final = int((re.findall(r'\d+', text[47]))[0])
#     Review_final = int(re.findall(r'\d+',text[56])[0])
#     Price_final = float(re.findall(r'\d+',text[22])[0])
    
#     # Category Scraping:
#     category = soup2.find('ul', class_ = 'breadcrumb')#.findAll('li')#.find("a", href = True)
#     text = list(category.descendants)
#     Category_final= text[16]
    
#     return Name_final, Rating_final, Price_final, UPC_final, Avai_final, Review_final, Category_final

    #      '; # of Review: ', Review_final, '; Category: ', Category_final)
    #data ={'Name': Name_final, 'Price': Price_final,
    #   'UPC': UPC_final, '# Avilable': Avai_final, '# of Review': Review_final, 'Category':Category_final}
    # DF =  pd.DataFrame(data)
    #print('Name: ', Name_final, '; Price: ', Price_final, '; UPC: ', UPC_final, '; # Avilable: ', Avai_final,
    #      '; # of Review: ', Review_final, '; Category: ', Category_final)
    # DF =  pd.DataFrame(data))
    

In [27]:
# Version 2:
def parse(URL):
                
    # Request and parsing the URL:      
    req2 = requests.get(URL)
    soup2 = BeautifulSoup(req2.text, "html.parser")
    
    # Name Scraping:
    Name = soup2.find('div', class_ = 'col-sm-6 product_main')
    text = list(Name.descendants)
    Name_final = text[2]
    
    # Rating Scraping:
    rating = soup2.find('div', class_= 'col-sm-6 product_main').find("p").find_next("p").find_next("p", class_= True)
    temp = rating['class']
    Rating_final = temp[1]
    if Rating_final == 'One':
        Rating_final = Rating_final.replace('One', '1')
    elif Rating_final == 'Two':
        Rating_final = Rating_final.replace('Two', '2')
    elif Rating_final ==  'Three':
        Rating_final = Rating_final.replace('Three', '3')   
    elif Rating_final ==  'Four':
        Rating_final = Rating_final.replace('Four', '4') 
    else:
        Rating_final = Rating_final.replace('Five', '5')  
    
    Rating_final = int(Rating_final)  
    
    
    # UPC, Availability and Review Scraping:
    UPC_Avai_Review_Price = soup2.find('table', class_ = 'table table-striped')
    text = list(UPC_Avai_Review_Price.descendants)
    UPC_final = text[6] 
    Avai_final = int((re.findall(r'\d+', text[47]))[0])
    Review_final = int(re.findall(r'\d+',text[56])[0])
    Price_final = float(re.findall(r'\d+',text[22])[0])
    
    # Category Scraping:
    category = soup2.find('ul', class_ = 'breadcrumb')#.findAll('li')#.find("a", href = True)
    text = list(category.descendants)
    Category_final= text[16]
    
    return Name_final, Rating_final, Price_final, UPC_final, Avai_final, Review_final, Category_final

In [28]:
# books_links:
URL3 = "https://books.toscrape.com/catalogue/"

API_ = []
books_links = []
for t in title_ref:
    API_ = '{}{}'.format(URL3,t)
    books_links.append(API_)

In [29]:
len(books_links)

1000

In [30]:
# Validating:
print(books_links[0])
parse(books_links[0])

https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html


('A Light in the Attic', 3, 51.0, 'a897fe39b1053632', 22, 0, 'Poetry')

In [31]:
# See the python file attached with all the multiprocessing code: multiprocessing_pool_books_case.py

## (5) SQLITE

In [54]:
# Loading DF into the data base:
db = "books.sqlite"
conn = sqlite3.connect(db)
cursor = conn.cursor()

In [55]:
# Saving DF in sqlite database:
DF.to_sql(name='DF2', con=conn)
# Note: Run it once. 
# After Loading in the db it does not need to be reloaded.

C:\Users\luisc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [62]:
cursor.execute('SELECT category, count(category) as count_books_per_categ, avg(rating) as avg_rating, sum(available) Sum_Avai  From DF2 Group By Category Order by avg_rating DESC' )
cursor.fetchall()

[('Novels', 1, 5.0, 9),
 ('Erotica', 1, 5.0, 15),
 ('Adult Fiction', 1, 5.0, 3),
 ('Christian Fiction', 6, 4.166666666666667, 55),
 ('Health', 4, 3.75, 49),
 ('Art', 8, 3.625, 72),
 ('Poetry', 19, 3.526315789473684, 235),
 ('Humor', 10, 3.4, 84),
 ('Spirituality', 6, 3.3333333333333335, 69),
 ('Young Adult', 54, 3.2962962962962963, 464),
 ('Historical Fiction', 26, 3.230769230769231, 194),
 ('Fiction', 65, 3.1846153846153844, 588),
 ('New Adult', 6, 3.1666666666666665, 49),
 ('Music', 13, 3.1538461538461537, 111),
 ('Religion', 7, 3.142857142857143, 60),
 ('Womens Fiction', 17, 3.1176470588235294, 91),
 ('Fantasy', 48, 3.0833333333333335, 372),
 ('Suspense', 1, 3.0, 8),
 ('Sports and Games', 5, 3.0, 10),
 ('Historical', 2, 3.0, 20),
 ('Autobiography', 9, 3.0, 57),
 ('Sequential Art', 75, 2.973333333333333, 686),
 ('History', 18, 2.9444444444444446, 181),
 ('Mystery', 32, 2.9375, 290),
 ('Science', 14, 2.9285714285714284, 105),
 ('Business', 12, 2.9166666666666665, 133),
 ('Food and Dri

In [53]:
conn.close()